<a href="https://colab.research.google.com/github/LeoneFabio/Egocentric-Vision/blob/main/Video_LLaVa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/PKU-YuanGroup/Video-LLaVA
!cd Video-LLaVA
!conda create -n videollava python=3.10 -y
!conda activate videollava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d
!pip install -e Video-LLaVA[train]

fatal: destination path 'Video-LLaVA' already exists and is not an empty directory.
/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Using cached flash_attn-2.7.2.post1.tar.gz (3.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)
  Cloning htt

In [ ]:
!pip install -q av
!pip install numpy
!pip install pillow
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-250hv6wd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-250hv6wd
  Resolved https://github.com/huggingface/transformers.git to commit e5fd865ebae062b7cf03a81b8c6affeb39f30bec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install protobuf==3.20.3

In [ ]:
from PIL import Image
import numpy as np
import av
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import torch
import os
torch.cuda.is_available()  # Dovrebbe restituire True se la GPU è attiva

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

True

In [ ]:

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


In [ ]:

'''
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)
'''
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
   quantization_config=quantization_config,
    device_map="auto"
)

# Carica il processore
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

# Definisci il prompt
prompt = "USER: <video>what do you see? ASSISTANT:"

video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
# Apri il video con PyAV
container = av.open(video_path)

# Campiona uniformemente 8 frame dal video
total_frames = container.streams.video[0].frames
# Campiona solo 2 frame dal video
indices = np.arange(0, total_frames, total_frames / 15).astype(int)  # Riduce i frame
clip = read_video_pyav(container, indices)


# Prepara input e genera risposta
inputs = processor(text=prompt, videos=clip, return_tensors="pt").to("cuda")
generate_ids = model.generate(**inputs, max_new_tokens=100)
print(processor.batch_decode(generate_ids, skip_special_tokens=True)[0])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

USER: what do you see? ASSISTANT: In the image, a baby is sitting on a bed and playing with a Wii remote.Ъ. The baby is wearing glasses and is holding the remote in their hands. The bed is covered with a white sheet, and there is a chair located next to the bed. The baby seems to be enjoying their time playing with the Wii remote.
